In [1]:
import gpytorch
import torch
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

from utils.data_loaders import get_wake_dataloaders

In [2]:
FACTORS_FOLDER = "discr_factors_x2_30_y-2_2_step0.125_TIstep0.01_CTstep0.01"
DATA_FOLDER = f"data/{FACTORS_FOLDER}/"
#MODEL_NAME = f"univariate_GP" #TODO make the name represent the structure or the parameters of the model
#BEST_MODEL_PATH = f"saved_models/{FACTORS_FOLDER}/{MODEL_NAME}.pt"
CONSIDER_WS = False #TODO
COORDS_AS_INPUT = False # multivariate setting

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"{DEVICE=}")

# hyperparameters
EPOCHS = 2 #TODO
LR = 0.1
BATCH_SIZE = 1

DEVICE=device(type='cpu')


In [4]:
valid_dataloader = None
train_dataloader, valid_dataloader, test_dataloader = get_wake_dataloaders(DATA_FOLDER,
                                                          consider_ws=CONSIDER_WS,
                                                          coords_as_input=COORDS_AS_INPUT,
                                                          train_perc=0.6,
                                                          test_perc=0.2,
                                                          validation_perc=0.2,
                                                          batch_size=BATCH_SIZE)

print(len(train_dataloader))

train_x, train_y = train_dataloader.dataset.inputs, train_dataloader.dataset.outputs

test_x, test_y = test_dataloader.dataset.inputs, test_dataloader.dataset.outputs
print("Train shapes: ", train_x.shape, train_y.shape)
print("Test shapes: ", test_x.shape, test_y.shape)

if valid_dataloader is not None:
    valid_x, valid_y = valid_dataloader.dataset.inputs, valid_dataloader.dataset.outputs
    print("Valid shapes: ", valid_x.shape, valid_y.shape)

grid_size = train_dataloader.dataset.num_cells
print(f"{grid_size=}")

5160
Train shapes:  torch.Size([5160, 2]) torch.Size([36986880])
Test shapes:  torch.Size([1720, 2]) torch.Size([12328960])
Valid shapes:  torch.Size([1720, 2]) torch.Size([12328960])
grid_size=7168


In [5]:
"""
REDUCE THE AMOUNT OF DATA
num_instances = 10 #TODO
train_x= torch.split(train_x, num_instances)[0]#.reshape(num_instances, grid_size, num_features)
train_y = torch.split(train_y,num_instances)[0]#.reshape(num_instances, grid_size)
print(train_x.shape, train_y.shape)"""

torch.Size([71680, 4]) torch.Size([71680])


# Sklearn

In [7]:
kernel = RBF(length_scale=1.0)
gp = GaussianProcessRegressor(kernel=kernel)
gp.fit(train_x, train_y)

'\nToo much data for simple Regressor\nkernel = RBF(length_scale=1.0)\ngp = GaussianProcessRegressor(kernel=kernel)\ngp.fit(train_x, train_y)\n'

# Gpytorch

In [8]:
class MultitaskGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(MultitaskGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.MultitaskMean(
            gpytorch.means.ConstantMean(), num_tasks=grid_size
        )
        self.covar_module = gpytorch.kernels.MultitaskKernel(
            gpytorch.kernels.RBFKernel(), num_tasks=grid_size, rank=1
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)


likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=grid_size)
model = MultitaskGPModel(train_x, train_y, likelihood)

In [9]:
# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=LR)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(EPOCHS):
    optimizer.zero_grad()
    output = model(train_x)
    loss = -mll(output, train_y)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f' % (i + 1, EPOCHS, loss.item()))
    optimizer.step()